
# Introduction

This workshop aims at generating Land Surface Temperature (LST) and other relevant products (Normalized Difference Vegetation Index,surface emissivity, and  fractional vegetation cover) for assessing urban climates (urban heat islands) from Landsat 8 imagery using open python geospatial tools.

We will use the city of Tallinn, Estonia's capital as our area of study. The image acquisition date is 27.07.2018, this day
is one of the series of days in the heatwave of 2018. LST is a widely used parameter to assess the impacts of heatwaves.


LST is the radiative skin temperature of the land surface. It is estimated from Top-of-Atmosphere brightness temperatures from the thermal bands of remote sensing imagery.Its estimation further depends on the surface albedo, the vegetation cover and the soil moisture.-- **Copernicus Global Land Services**

Several algorithms exist for calculating LST from remote sensing images, examples are split-window, dual-angle, and single channel.


In this workshop we will go through the workflow for calculating LST using the **single channel** approach by Jiménez-Muñoz et. al. 2009. 



## Data

Landsat imagery -(Green-3,Red-4, Near Infrared-5, and Thermal-10 bands)

The bands (3,4, and 5) are from landsat collection 2 and they are already processed to surface reflectance.

The thermal band is from landsat collection 1.

1 km Population Grids



## Key Python libraries

__[Intake](https://pypi.org/project/intake/)__

__[Rasterio](https://pypi.org/project/rasterio/)__

__[Earthpy](https://pypi.org/project/earthpy/)__

__[Numpy](https://numpy.org/)__

__[Geopandas](https://geopandas.org/index.html)__

__[Rasterstats](https://pypi.org/project/rasterstats/)__

__[Geocube](https://pypi.org/project/geocube/)__

__[Matplotlib](https://matplotlib.org/stable/index.html#)__

## Workflow

![WORKFLOW](workflow.png)




## Outputs
The final outputs for this workshop will be a 1 Km grided map of the LST for the city of Tallinn, Estonia and a datacube
of LST and NDVI.




In [1]:
#import the needed packages

import intake

import numpy as np
import pandas as pd

import rasterio as rio
from rasterio.crs import CRS


import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
from geocube.api.core import make_geocube
from rasterstats import zonal_stats
from affine import Affine



import matplotlib.pyplot as plt
import matplotlib.colors as mcl

import warnings
warnings.filterwarnings('ignore')

The relevant data for this workshop is accessible through the intake catalog in the Data folder.

The catalog contains a stack of the relevant bands (3,4,5, and 10), 1 km grids, population data at 1 km and atmospheric parameters.

The data is accessible at this __[Data Repo](https://owncloud.ut.ee/owncloud/index.php/s/RJefY58gpaNakyg)__ 




A quick look of the extent of the satellite image and the study area

![Study Area](area.png)

In [2]:
# open data catalog
cat = intake.open_catalog('SC52_catalog.yaml')

cat.discover()


# View components of catalog

list (cat)



['atm_params', 'grids_1000_utm', 'stacked_bands']



We will access the stacked_bands in the catalog, have an overview and fetch the individual bands we will be using.



In [ ]:
stack = cat.stacked_bands.read() #read stacked bands in the catalog

stack

Now, we will proceed to selecting the individual bands (arrays) in the stack as numpy arrays.

We will do that by using indices. The bands are arranged in the order (Green-3,Red-4, Near Infrared-5, and Thermal-10 bands) which match the indices 0, 1, 2, and 3 in the stack, respectively.

In [ ]:


b4 = np.array(stack[1])#red band

b5= np.array(stack[2]) # NIR band

b10 = np.array(stack[3]) #thermal band




Now we will do same for the other datasets in the catalog.

we will read and have a preview for each.

In [ ]:
grids = cat.grids_1000_utm.read()  #pop grids

grids.head(5) #view first ten rows 

In [ ]:
#quick plot of grids

grids.boundary.plot()

In [ ]:
atm_params = cat.atm_params.read()  #atm corr params

atm_params #view atmospheric parameters

Since we will be visualizing our intermediate and final outputs,we will create custom color maps for the parameters of interest. 

we will use standard colors that are used in representing these parameters usually.

We will use the LinearSegmentedColormap in matplotlib by passing a list of colours.


In [ ]:
#create custom colour map
lst_cmap = mcl.LinearSegmentedColormap.from_list("", ["blue","lightskyblue","yellow","orange",'red'])

#create custom colour map
ndvi_cmap = mcl.LinearSegmentedColormap.from_list("", ["red","yellow","greenyellow","limegreen",'forestgreen','darkgreen'])


#create custom colour map
pop_cmap = mcl.LinearSegmentedColormap.from_list("", ["khaki","yellow","orange","darkgoldenrod","saddlebrown","maroon"])

Now we are set to go through the workflow having read all the needed data.

## **Calculate NDVI**

NDVI is the Normalized Difference Vegetation Index. It is arguably the most used index for vegetation assessment. 

Since the late 1960s researchers have used red and near-infrared light to estimate vegetation abundance using the NDVI.

The index is also used for urban climate studies.

NDVI values range from **-1 to +1**. Negative values signify bare lands while positive values signify vegetated surfaces.

The higher the positive value, the 'healthier' the vegetation or the more green the area. Urban regions have lower positive values usually.

The formula is given by:



$NDVI = \frac{\rho_{NIR} - \rho_{Red}}{\rho_{NIR} + \rho_{Red}}$

where $ \rho_{NIR} $ is the near-infrared band and $ \rho_{Red} $ is the red band.

the red band for landsat 8 is band 4 and the near-infrared is band 5


In [ ]:


ndvi = (b5.astype(float)-b4.astype(float))/(b5+b4) # calculate NDVI



ndvi

Let us visualize our first intermediate results-NDVI.

We will use the earthpy.plot which is a plot function in earthy built on matplotlib.

We will also use this first visualisation to have an idea of the coverage of the image and study area.

In [ ]:
#visualize NDVI output

ep.plot_bands(ndvi, cmap=ndvi_cmap, vmin=-0.2,vmax=1)




**Calculate Fractional Vegetation Cover (FVC)**

Fractional vegetation cover (FVC) is the ratio of vertically projected area of vegetation to the total surface extent.

It is a controlling factor in transpiration, photosynthesis, global climate changes and other terrestrial processes and climate models.

FVC is essential for calculating the land surface emmisivity. It can be estimated from the NDVI according to the following formula
 
$$ FVC =  \frac{NDVI - NDVI_{soil}  }  {NDVI_{veg} -  NDVI_{soil}}$$ 

where $NDVI_{soil}$ is given as **0.2** and  $NDVI_{veg}$ is given as **0.5** (Sobrino et al. 2004; Yu et al. 2014)



In [ ]:
#calculate the fvc

fvc = (ndvi-0.2)/(0.5-0.2)

In [ ]:
ep.plot_bands(fvc, cmap=ndvi_cmap,vmin=-1.2, vmax=2.7)

The emissivity (ε) of a material is the relative ability of its surface to emit heat by radiation.

The strength of the energy emitted depends on both the temperature of the surface and how efficiently it can emit radiation.

The emissivity of most natural Earth surfaces is a unitless quantity and ranges between approximately 0.6 and 1.0, but surfaces with emissivities less than 0.85 are typically restricted to deserts and semi-arid areas. Vegetation, water and ice have high emissivities above 0.95 in the thermal infrared wavelength range- __[NASA](https://www.jpl.nasa.gov/images/nasa-spacecraft-maps-earths-global-emissivity)__


To estimate land surface emissivity (LSE) of mixed pixels using the equation from (Sobrino et al. 2004):
    
    
$$ \varepsilon =  \varepsilon_s(1-FVC)+ \varepsilon_v(FVC)  $$

where $ \varepsilon_s $  and $ \varepsilon_v $  are emissivity values for soil and vegetation, respectively.


The emissivity values for soil and vegetation estimated at 0.97 and 0.99 respectively.


We will go through the image and calculate the emissivity for each pixel using the NDVI threshold method.

For

$NDVI ≤ 0.2$ : $LSE = 0.97$ - bare soils (low vegetaton)



$NDVI ≥ 0.5$ : $LSE = 0.99$ - mostly (healthy) vegetation


$0.2 < NDVI < 0.5$ : $LSE = 0.971*(1-FVC)+0.987*FVC$

In [ ]:

emi = []

for row in range(len(ndvi)):                         #each row in the ndvi array
    
    emrow=[]                                              #emissivity of the row
    
    for i in range(len(ndvi[row])):                  #for each pixel in the row
        
        if ndvi[row][i] ==np.nan:
            emrow.append(np.nan)
            
        elif ndvi[row][i]<= 0.2:                    
            emrow.append(0.97)
            
        elif ndvi[row][i]>= 0.5:
            emrow.append(0.99)
            
        else:
            emrow.append(0.971*(1-fvc[row][i])+(0.987*fvc[row][i]))
    emi.append(emrow)
    
    
emiss = np.asarray(emi) #convert the emi to np array    

In [ ]:
ep.plot_bands(emiss, cmap=ndvi_cmap)

## **Calculate LST**

Now, we will move on to process the thermal band to calculate the LST.

we will calculate the at sensor radiance (convert digital numbers to radiance); brightness temperature and two parameters based on Planck's functions which are needed to calculate LST.

The LST formular is given by :
    

    
$ lst = \gamma [\frac{1}{\varepsilon} (\psi_1*toa + \psi_2) + \psi_3] + \delta\\ $

$ \gamma $ and $ \delta$ which are parameters based on Planck's funtions

$ \varepsilon$ is emissivity

$\psi_1$, $\psi_2$ , and $\psi_3$ are atmospheric functions

**toa** is the top of atmosphere radiance

In [ ]:
#calculate at top of atmosphere radiance


toa = (b10.astype(float)*0.0003342)+0.1 #change mult constant if not L8


Now we will proceed to calculate the at-sensor brightness temperature (bt) using :

    
$bt = \frac{K_2}{In  (\frac{K_1}{toa}+1)} $
    

Also we will canulate $ \gamma $ and $ \delta$ which are parameters based on Planck's funtions:


$ \gamma = \frac {bt^2}  {K_2  *  toa}  $

$ \delta =  bt - \frac {bt^2}{K_2}  $

where $K_1$ and $K_2$ are thermal band constants provided in the metadata of the image.

For Landsat 8 $K_1$ = 774.8853 and $K_2$ = 1321.0789

In [ ]:
#calculate at sensor brightness temperature and convert to degree cel

bt = (1321.0789 / np.log((774.8853 / toa) + 1)) - 273.15


#calculate delta and gamma -- parameters based on Plank's functions



gamma = (bt*bt)/(1321.0789*toa) 

delta = bt - ((bt*bt)/1321.0789) 






We calculate atmospheric functions $\psi_1$, $\psi_2$ , and $\psi_3$

from MODTRAN upwelling and downwelling estimations using the relationships in (Sobrino and Jiménez-Muñoz, 2010)
 
 
 
$ \psi_1 = \frac{1}{\tau} $

$ \psi_2 = −𝐿^\downarrow − \frac{𝐿^\uparrow}{\tau}$
   
$ \psi_3 = 𝐿^\downarrow $



where $\tau$ is the atmospheric transmissivity, $𝐿^\uparrow$ is the upwelling atmospheric radiance, and $𝐿^\downarrow$ is the downwelling atmospheric radiance. These parameters can be obtained from the online calculator at https://atmcorr.gsfc.nasa.gov/ by providing date and time of image acquisition found in the metadata

In [ ]:
#a look at the atm param

atm_params

In [ ]:
#calcultate atm functions 




transmission = float(atm_params[0][atm_params[0].find(':')+1:].strip())

upwelling_radiance= float(atm_params[1][atm_params[1].find(':')+4:atm_params[1].find('W')-1].
                          strip())

downwelling_radiance= float(atm_params[2][atm_params[2].find(':')+2:atm_params[2].find('W')-1].
                            strip())



w1 = 1/transmission

w2 =(-1*downwelling_radiance) - (upwelling_radiance/transmission)

w3 = downwelling_radiance





print ('transmission:',transmission,' upwelling_radiance:',upwelling_radiance,' downwelling_radiance:',downwelling_radiance)

Up until now we have generated all the required parameters to calculate the Land Surface Temperature.

Now we can go ahead and calculate the lst

The LST formular is given by :
    

    
$ lst = \gamma [\frac{1}{\varepsilon} (\psi_1*toa + \psi_2) + \psi_3] + \delta\\ $

In [ ]:
lst = gamma*((((1/emiss)*((w1*toa)+w2)) + w3))+delta

In [ ]:
ep.plot_bands(lst, cmap=lst_cmap ,
              scale=False,
            vmin=15, vmax=40)
plt.show

## Extration of LST and NDVI to grids

Now, we will extract the data for our area of interest using the 1km population grids. 

we will extract the NDVI and the LST for each grid using zonal statistics.

To perform the zonal statistics, we will use rasterstats. we will require the transformation information from the imagery as one of the arguments for the zonal statistics.

In [ ]:
#fetch transformation info and use the affine method

T_tup = stack.transform

print(T_tup)

The Affine method accepts 6 positional arguments. We can either copy the values we printed in the previous 
cell or access each element in the tuple and pass them to the Affine method. Copying is simple.

In [ ]:
#create affine transformation using affine method

trans = Affine(T_tup[0], T_tup[1], T_tup[2], T_tup[3], T_tup[4], T_tup[5])

trans

In [ ]:
#zonal stats for NDVI 

sampled_ndvi = zonal_stats(grids,                       #define polygon feature (grids)
                            ndvi,                       #input raster
                            stats='mean',               #statistic of interest
                            geojson_out=True,           #output as geojson
                            affine = trans)             #transformation



#Zonal stats for LST

sampled_lst = zonal_stats(grids, 
                          lst,
                          stats='mean',
                          geojson_out=True,
                          affine = trans)




The output of the zonal stats is  geojson. We will convert it to GeoDataFrame.

In [ ]:
#sampled_ndvi geojson to GeoDataFrame
sampled_ndvi_gdf = gpd.GeoDataFrame.from_features(sampled_ndvi)




#sampled_lst geojson to GeoDataFrame
sampled_lst_gdf = gpd.GeoDataFrame.from_features(sampled_lst)

In [ ]:
sampled_ndvi_gdf.head(5)

In [ ]:
sampled_lst_gdf.head(5)

Let us put the sampled ndvi and lst in the grids dataframe.

In [ ]:
grids['ndvi'] = sampled_ndvi_gdf['mean'].round(2)    #round extracted values to 2 dp


grids['lst'] = sampled_lst_gdf['mean'].round(2)



In [ ]:
grids.head(5)

Now our grids dataframe has all the parameters of interest. 

Through out the workflow, we worked in UTM reference system, however we will like to our final exports to be in Estonian reference system - EPSG: 3301

In [ ]:
grids = grids.to_crs('EPSG:3301') #reproject from utm to Estonian projected CRS

In [ ]:
#Quick look at our grid data

grids.head(10)

With all our desired information in one geodataframe, we can now visualize each parameter in the study area.

Let's start with plotting the NDVI and the LST side by side. 

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(15,5),dpi=600) #create a figure with two subplots

ax1 , ax2 = axes.flatten() #flatten the axes to get individual matplotlib ax


#plot the LST in the first ax

ax2 = grids.plot(column='lst',     #define parameter of interest
                 cmap=lst_cmap,    
                 ax=ax2,           #set the ax 
                 legend=True,
                 legend_kwds={'shrink': 0.7})

#some extra tweaks to enhance the presentation

ax2.ticklabel_format(style='plain')
ax2.figure.axes[-1].tick_params(labelsize=8)
ax2.set_title('Land Surface Temperature (°C)')


ax1 = grids.plot(column='ndvi',     #define parameter of interest
                 cmap=ndvi_cmap,    
                 ax=ax1,           #set the ax 
                 legend=True,
                 legend_kwds={'shrink': 0.7})

ax1.ticklabel_format(style='plain')
ax1.figure.axes[-1].tick_params(labelsize=8)
ax1.set_title('NDVI')

plt.savefig('demo.png', dpi=600,
        transparent=False,
        frameon=None)

Finally we will make a geocube (data cube) from the grids dataframe and export using geocube.

The geocube library is built on rasterio.

This is a convenient way to export both parameters in one shot. Another advantage is that you do not have to define meta parameters as you would when using rasterio

In [ ]:
#make cube and export
cube = make_geocube(vector_data=grids, resolution=(1000, 1000))
    
    
cube.rio.to_raster('tallinn_lst_ndvi.tif')

### Some resources:

__[Github repo](https://github.com/LandscapeGeoinformatics/EGU_2021_lgeo_workshops)__ for this workshop.
    


__[How to download data](https://lta.cr.usgs.gov/sites/default/files/LS_C2_Help_122020.pd)__ 

Sobrino, J.A.; Jimenez-Muoz, J.C.; Soria, G.; Romaguera, M.; Guanter, L.; Moreno, J.;
Plaza, A.; Martinez, P., "Land Surface Emissivity Retrieval From Different VNIR and TIR Sensors," in Geoscience and Remote Sensing, IEEE Transactions on , vol.46, no.2, pp.316-
327, Feb. 2008 doi: 10.1109/TGRS.2007.904834
    
J. C. Jimenez-Munoz, J. Cristobal, J. A. Sobrino, G. Soria, M. Ninyerola and X. Pons, 
"Revision of the Single-Channel Algorithm for Land Surface Temperature Retrieval From Landsat Thermal-Infrared Data," 
in IEEE Transactions on Geoscience and Remote Sensing, vol. 47, no. 1, pp. 339-349, Jan. 2009, doi: 10.1109/TGRS.2008.2007125.
        
